In [1]:
!pip install transformers datasets --quiet

In [2]:
from datasets import Dataset
import pandas as pd
import torch

In [4]:
with open("miTexto.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

In [5]:
lines = [line.strip() for line in lines if line.strip()]
#lines = lines[:500]  # ← limitar para evitar sobrecarga

df = pd.DataFrame({"text": lines})
dataset = Dataset.from_pandas(df)

In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [7]:
from transformers import (
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

In [8]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [11]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:
training_args = TrainingArguments(
    output_dir="./gpt2-discard",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_steps=30,
    report_to="none",
    save_strategy="no",  # ← no guarda nada
    fp16=torch.cuda.is_available()
)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-20-2113583967>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
30,4.080500
60,3.520200
90,3.171100
120,2.790500
150,2.675700
180,2.535000
210,2.386900
240,2.379600


TrainOutput(global_step=240, training_loss=2.9424400965372723, metrics={'train_runtime': 26.1419, 'train_samples_per_second': 36.34, 'train_steps_per_second': 9.181, 'total_flos': 62056857600000.0, 'train_loss': 2.9424400965372723, 'epoch': 10.0})

In [21]:
model.save_pretrained("./gpt2-mi-modelin")
tokenizer.save_pretrained("./gpt2-mi-modelin")

('./gpt2-mi-modelin/tokenizer_config.json',
 './gpt2-mi-modelin/special_tokens_map.json',
 './gpt2-mi-modelin/vocab.json',
 './gpt2-mi-modelin/merges.txt',
 './gpt2-mi-modelin/added_tokens.json')

In [22]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

model = GPT2LMHeadModel.from_pretrained("./gpt2-mi-modelin")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-mi-modelin")

In [31]:
from transformers import pipeline

#generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generator = pipeline(
    "text-generation",
    model="datificate/gpt2-small-spanish",
    tokenizer="datificate/gpt2-small-spanish"
)

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cuda:0


In [32]:
prompt = "En la mañana"
output = generator(prompt, max_new_tokens=200, temperature=0.7, top_k=50)
print(output[0]["generated_text"])

En la mañana del domingo 11 de octubre, el comandante de la 2.ª Brigada de Infantería, John R. McGarry, condujo a la Brigada de Infantería de Reserva de los Estados Unidos (USAC) en Freetown, Carolina del Norte, y se unió al cuerpo de Marines de los Estados Unidos (USAC) y al Cuerpo de Marines de los Estados Unidos (USAC) en la Base Aérea Naval de la Base Naval de la Reina Isabel, Florida. Durante la operación, McGarry y el resto del personal de la unidad recibieron el nombre de T-34.

El 12 de octubre, la 1.ª Brigada de Infantería de Marines de los Estados Unidos (USAC) tomó el control de la Base Aérea Naval de la Reina Isabel, Florida. El 17 de octubre, el Cuerpo de Marines de los Estados Unidos (USAC) comenzó a desembarcar a 2.500 marines de la Infantería de Marina y el Cuerpo de Marines de los Estados Unidos (USAC) en la Base
